In [ ]:
import numpy as np
from psycopg2 import connect
import pandas as pd
import pm4py
import numpy as np
import pandasql as ps
from pm4py.objects.conversion.log import converter as log_converter
from scipy.stats import variation
from scipy import stats
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
from pm4py.visualization.dfg import visualizer as dfg_visualization
from pm4py.statistics.eventually_follows.log import get as efg_get
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import sklearn.preprocessing as sk
from scipy.stats import chi2_contingency
import math
import statistics
import pingouin as pg
import graphviz
from statsmodels.stats import multitest
from statsmodels.stats.contingency_tables import SquareTable as ST

In [ ]:
df = pd.read_csv("Logs/MIMIC_Log_Raw.csv")

In [ ]:
df = df.rename({"hadm_id":"case:hadm_id"}, axis=1)

In [ ]:
label_list = ["Dialysis - CRRT", "Hemodialysis", "Non-invasive Ventilation", "Invasive Ventilation"]

In [ ]:
proc_re = df.loc[df["concept:name"].isin(label_list)]

In [ ]:
proc_re["time:timestamp"] = proc_re["time:timestamp"].apply(lambda x: pd.to_datetime(x))
proc_re["endtime"] = proc_re["endtime"].apply(lambda x: pd.to_datetime(x))

In [ ]:
hadms = list(proc_re["case:hadm_id"].unique())

In [ ]:
proc_re = proc_re[["case:hadm_id", "time:timestamp", "endtime", "ordercategoryname", "category", "concept:name"]]

In [ ]:
marker_list = ["Creatinine (serum)", "BUN", "Anion gap", "Hematocrit (serum)", "Heart Rate", "Respiratory Rate", "O2 saturation pulseoxymetry", "Non Invasive Blood Pressure systolic", "Non Invasive Blood Pressure diastolic"]

In [ ]:
meas = df.loc[df["label"].isin(marker_list)]

In [ ]:
meas = meas.sort_values(["case:hadm_id", "time:timestamp"])

In [ ]:
meas_log = meas.copy()

In [ ]:
meas_log["concept:name"] = "Measurement"

In [ ]:
meas_log = meas_log.drop_duplicates(["case:hadm_id", "time:timestamp", "concept:name"])

In [ ]:
meas_log = meas_log.loc[meas_log["case:hadm_id"].isin(hadms)]

In [ ]:
#make endtime represented as second event
proc_re = proc_re.reset_index().drop("index", axis=1)

In [ ]:
rows_list = []

for index, row in proc_re.iterrows():
    r = row
    r = r.drop("time:timestamp", axis="index")
    r = r.rename({"endtime":"time:timestamp"})
    r["concept:name"] = "END " + r["concept:name"]
    rows_list.append(r)

to_concat = pd.DataFrame(rows_list)

In [ ]:
proc_start = proc_re

In [ ]:
proc_start["concept:name"] = "START " + proc_start["concept:name"]

In [ ]:
proc_start.drop("endtime", axis=1, inplace=True)

In [ ]:
proc_start_end = pd.concat([proc_start, to_concat])

In [ ]:
proc_start_end = proc_start_end.sort_values(["case:hadm_id", "time:timestamp"])
proc_start_end = proc_start_end.reset_index().drop("index", axis=1)

In [ ]:
proc_start_end

In [ ]:
meas_log = meas_log.reset_index().drop("index", axis=1)

In [ ]:
chart_proc_start_end = pd.concat([proc_start_end, meas_log])

In [ ]:
chart_proc_start_end = chart_proc_start_end.sort_values(["case:hadm_id", "time:timestamp"])

In [ ]:
meas["charttime"] = meas["charttime"].apply(lambda x: pd.to_datetime(x))

In [ ]:
#separate into category/value label

In [ ]:
labels = meas["label"].unique()

In [ ]:
#Link measurments to activities
time_from_last_proc_event = pd.to_datetime(0)
time_from_next_proc_event = pd.to_datetime(290, unit="Y")
rows_list = []
for index, row in proc_start_end.iterrows():
    row_to_add = {}
    hadm_id = row["case:hadm_id"]
    act = row["concept:name"]
    if index == len(proc_start_end) - 1:
        time_from_next_proc_event = pd.to_datetime(290, unit="Y")
    elif proc_start_end.iloc[index+1]["case:hadm_id"] == hadm_id:
        time_from_next_proc_event = proc_start_end.iloc[index+1]["time:timestamp"]
    else:
        time_from_next_proc_event = pd.to_datetime(290, unit="Y")
    is_start = "START" in act
    meas_p = meas.loc[meas["case:hadm_id"] == hadm_id]
    if is_start:
        meas_p["time_delta"] = row["time:timestamp"] - meas_p["charttime"]
        meas_p = meas_p.loc[(meas_p["time_delta"] > pd.Timedelta(0)) & (meas_p["charttime"] > time_from_last_proc_event)]
        row_to_add["concept:name"] = "Measurement"
    else:
        meas_p["time_delta"] = meas_p["charttime"] - row["time:timestamp"] 
        meas_p = meas_p.loc[(meas_p["time_delta"] > pd.Timedelta(0)) & (meas_p["charttime"] < time_from_next_proc_event)]
        row_to_add["concept:name"] = "Measurement"
    row_to_add["case:hadm_id"] = hadm_id
    row_to_add["time:timestamp"] = None
    for lab in labels: 
        lab_val = meas_p.loc[meas_p["label"] == lab]
        try:
            closest_val_index = lab_val["time_delta"].idxmin()
            closest_val_row = lab_val.loc[closest_val_index]
            row_to_add[lab] = closest_val_row["valuenum"]
            if row_to_add["time:timestamp"] is None:
                row_to_add["time:timestamp"] = closest_val_row["charttime"]
            if is_start:
                if closest_val_row["charttime"] > row_to_add["time:timestamp"]:
                    row_to_add["time:timestamp"] = closest_val_row["charttime"]
            else: 
                if closest_val_row["charttime"] < row_to_add["time:timestamp"]:
                    row_to_add["time:timestamp"] = closest_val_row["charttime"]
        except:
            row_to_add[lab] = None
    if index == len(proc_start_end) -1:
        time_from_last_proc_event = pd.to_datetime(0)
    elif proc_start_end.iloc[index+1]["case:hadm_id"] == hadm_id:
        time_from_last_proc_event = row["time:timestamp"]
    else:
        time_from_last_proc_event = pd.to_datetime(0)
    if row_to_add["time:timestamp"] is not None:
        rows_list.append(row_to_add)
meas_events = pd.DataFrame(rows_list)

In [ ]:
final_log = pd.concat([proc_start_end, meas_events])

In [ ]:
final_log = final_log.sort_values(["case:hadm_id", "time:timestamp"])

In [ ]:
final_log.to_csv("Logs/MIMIC_Log.csv")